In [305]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [306]:
import os
import numpy as np
import pandas as pd

In [307]:
def get_row_data(google_sheet_url, sheet):
  # Get data
  wb = gc.open_by_url(google_sheet_url)
  sheet_value = wb.worksheet(sheet)
  data = sheet_value.get_all_values()

  # create DataFrame
  df = pd.DataFrame(data)
  df.columns = df.iloc[0]
  df = df.iloc[1:]

  return df

In [308]:
def get_code(module, option, topic, subtopic, subsubtopic):

  if subtopic == 'NA':
    code = f'{module} {option} {topic}'
    return code
  elif subsubtopic == 'NA':
    code = f'{module} {option} {topic} {subtopic}'
    return code
  else:
    code = f'{module} {option} {topic} {subtopic} {subsubtopic}'
    return code

# Read data

In [309]:
PATH = '/content/drive/MyDrive/BOT COMUNICACIÓNES ECCI/'

TABLE_OF_CONTENTS = 'URL GOOGLE SHEETS'
FORM_ANSWERS = 'URL GOOGLE SHEETS'

In [310]:
dfTab3 = get_row_data(TABLE_OF_CONTENTS, 'TABLA 3')

dfAnswers = get_row_data(FORM_ANSWERS, 'Respuestas de formulario 1')

# Preprocesing

In [311]:
dfAnswers['MÓDULO'] = dfAnswers['MÓDULO'].replace(
    {'Pregrado estudiante antiguo':'1',
     'Posgrado estudiante antiguo':'2',
     'Aspirante':'3',
     'Educación continua':'4',
     'Centro de lenguas':'5'})

In [312]:
dfTab3['Index'] = dfTab3.index

# Analysis

In [316]:
for i in range(len(dfAnswers)):
  module, option, topic = dfAnswers[['MÓDULO', 'OPCIÓN DEL MÓDULO', 'TEMA']].iloc[i]
  subtopic, subsubtopic = dfAnswers[['SUBTEMA', 'SUBSUBTEMA']].iloc[i]
  date, form_type, update, person = dfAnswers[['Marca temporal', 'TIPO DE SOLICITUD', 'ACTUALIZACIÓN', 'RESPONSABLE']].iloc[i]

  code = get_code(module, option, topic, subtopic, subsubtopic)

  query_ids = code.split(' ')

  size_list = len(query_ids)

  if size_list == 3:
    module = query_ids[0]
    option = query_ids[1]
    topic = query_ids[2]

    index_value = dfTab3.index[(dfTab3['ID - MODULO'] == module) & (dfTab3['OPCIÓN'] == option) & (dfTab3['TEMA'] == topic)].tolist()
    index_value = index_value[0]

    dfTab3.at[index_value, 'FECHA'] = date
    dfTab3.at[index_value, 'TIPO DE SOLICITUD'] = form_type
    dfTab3.at[index_value, 'ACTUALIZACIÓN'] = update
    dfTab3.at[index_value, 'RESPONSABLE'] = person

  elif size_list == 4:
    module = query_ids[0]
    option = query_ids[1]
    topic = query_ids[2]
    subtopic = query_ids[3]
    
    index_value = dfTab3.index[(dfTab3['ID - MODULO'] == module) & (dfTab3['OPCIÓN'] == option) & (dfTab3['TEMA'] == topic) & (dfTab3['SUBTEMA'] == subtopic)].tolist()
    index_value = index_value[0]
    
    dfTab3.at[index_value, 'FECHA'] = date
    dfTab3.at[index_value, 'TIPO DE SOLICITUD'] = form_type
    dfTab3.at[index_value, 'ACTUALIZACIÓN'] = update
    dfTab3.at[index_value, 'RESPONSABLE'] = person

  elif size_list == 5:
    module = query_ids[0]
    option = query_ids[1]
    topic = query_ids[2]
    subtopic = query_ids[3]
    subsubtopic = query_ids[4]
    
    index_value = dfTab3.index[(dfTab3['ID - MODULO'] == module) & (dfTab3['OPCIÓN'] == option) & (dfTab3['TEMA'] == topic) & (dfTab3['SUBTEMA'] == subtopic) & (dfTab3['SUBSUBTEMA'] == subsubtopic)].tolist()
    index_value = index_value[0]
    
    dfTab3.at[index_value, 'FECHA'] = date
    dfTab3.at[index_value, 'TIPO DE SOLICITUD'] = form_type
    dfTab3.at[index_value, 'ACTUALIZACIÓN'] = update
    dfTab3.at[index_value, 'RESPONSABLE'] = person

['1', '2', '3']
['1', '1', '1']
['1', '2', '4']
['1', '2', '8']
['1', '2', '10']
['1', '1', '1']


# Save

In [317]:
columns_drop = ['Index']

for i in columns_drop:
    dfTab3.drop([i], axis = 1, inplace=True)

In [318]:
dfTab3.to_excel(PATH + 'actualización.xlsx', index=False)